In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
lonStep_1m = 0.0000105
latStep_1m = 0.0000090201
# 划分栅格
class RoadGrid:
    def __init__(self, label, grid_size):
        length = grid_size*latStep_1m
        width = grid_size*lonStep_1m
        self.length = length
        self.width = width
        def orginal_plot(label):
            tr = np.max(label,axis=0)
            tr[0]+=25*lonStep_1m
            tr[1]+=25*latStep_1m
            # plot(label[:,0], label[:,1], 'b,')
            bl = np.min(label,axis=0)
            bl[0]-=25*lonStep_1m
            bl[1]-=25*latStep_1m

            # width = (tr[1]-bl[1])/100
            # wnum =int(np.ceil((tr[1]-bl[1])/length))
            # for j in range(wnum):
                # hlines(y = bl[1]+length*j, xmin = bl[0], xmax = tr[0], color = 'red')

            # lnum = int(np.ceil((tr[0]-bl[0])/width))
            # for j in range(lnum):
                # vlines(x = bl[0]+width*j, ymin = bl[1], ymax = tr[1], color = 'red')
            return bl[0], tr[0], bl[1], tr[1]

        xl,xr,yb,yt = orginal_plot(label)
        self.xl = xl
        self.xr = xr
        self.yb = yb
        self.yt = yt
        gridSet = set()
        grid_dict = {}
        self.grid_dict = {}
        for pos in label:
            lon = pos[0]
            lat = pos[1]

            m = int((lon-xl)/width)
            n = int((lat-yb)/length)
            if (m,n) not in grid_dict:
                grid_dict[(m,n)] = []
            grid_dict[(m,n)].append((lon, lat))
            gridSet.add((m,n))
        # print len(gridSet)
        gridlist = list(gridSet)

            
            
        grid_center = [tuple(np.mean(np.array(grid_dict[grid]),axis=0)) for grid in gridlist]


        # for gs in gridSet:
            # xlon = xl+gs[0]*width
            # ylat = yb+gs[1]*length
            # bar(xlon,length,width,ylat,color='#7ED321')
        self.gridlist = gridlist

        self.grids = [(xl+i[0]*width,yb + i[1]*length) for i in grid_dict.keys()] # 左下角的点
        self.grid_center = grid_center
        self.n_grid = len(self.grid_center)
        self.grid_dict = grid_dict

    def transform(self, label, sparse=True):
        def one_hot(idx, n):
            a = [0] * n
            a[idx] = 1
            return a
        grid_pos = [self.gridlist.index((int((i[0]-self.xl)/self.width),int((i[1]-self.yb)/self.length))) for i in label]
        if sparse:
            grid_pos = np.array([one_hot(x, len(self.gridlist)) for x in grid_pos], dtype=np.int32)
        return grid_pos
    
def rad(d):
    return d * math.pi / 180.0

# 地理坐标系：为球面坐标。 参考平面地是椭球面，坐标单位：经纬度；
# 投影坐标系：为平面坐标。参考平面地是水平面，坐标单位：米、千米等；
# 地理坐标转换到投影坐标的过程可理解为投影。（投影：将不规则的地球曲面转换为平面）

# 目前国内主要有三种地理坐标系
# 1、WGS84坐标系：即地球坐标系（World Geodetic System），国际上通用的坐标系。
# 设备包含的GPS芯片或者北斗芯片获取的经纬度一般都是为WGS84地理坐标系，目前谷歌地图采用的是WGS84坐标系（中国范围除外）。
# 2、GCJ02坐标系：即火星坐标系，国测局坐标系。是由中国国家测绘局制定。由WGS84坐标系经加密后的坐标系。谷歌中国和搜搜中国采用。
# 3、BD09坐标系：百度坐标系，GCJ02坐标系经加密后的坐标系。

# 投影：墨卡托投影、高斯-克吕格 (Gauss-Krüger) 投影
# 感兴趣的同学可以在https://desktop.arcgis.com/zh-cn/arcmap/10.3/guide-books/map-projections/list-of-supported-map-projections.htm深入了解

# gps两点间距离（单位为米）
def distance(true_pt, pred_pt):
    lat1 = float(true_pt[1])
    lng1 = float(true_pt[0])
    lat2 = float(pred_pt[1])
    lng2 = float(pred_pt[0])
    radLat1 = rad(lat1)
    radLat2 = rad(lat2)
    a = radLat1 - radLat2
    b = rad(lng1) - rad(lng2)
    s = 2 * math.asin(math.sqrt(math.pow(math.sin(a/2),2) +
    math.cos(radLat1)*math.cos(radLat2)*math.pow(math.sin(b/2),2)))
    s = s * 6378.137
    s = round(s * 10000) / 10
    return s

In [5]:
# 读取数据
# sf = pd.read_csv("ft_ns.csv").head(500)
sf = pd.read_csv("../Trajectory data/2G_data.csv", encoding = 'gb2312').head(500)
#sf = sf.append(_)
#匹配工参
# sf_eng = pd.read_csv("ft_ns_bs_fix.csv")
sf_eng = pd.read_csv("../Trajectory data/2G_gongcan.csv", encoding = 'gb2312')

In [7]:
for i in range(1, 9):
    sf = sf.merge(sf_eng, left_on = ['mnc_%d'% i, 'lac_%d' % i, 'cid_%d' % i,], 
                  right_on = ['mnc','lac','cid'], how = 'left', suffixes = ('', '%d' % i))
sf = sf.fillna(-999)
#过滤无用数据
for i in range(1, 9):
    sf = sf[(sf['mcc_%d' % i] == 460) | (sf['mcc_%d' % i] == -1)]
sf = sf.reset_index(drop=True)

KeyError: 'mnc'

In [4]:
feature_cloumns_name = [ 'mnc_1','lac_1', 'cid_1','ss_1',
                        'mnc_2','lac_2', 'cid_2','ss_2',
                        'mnc_3','lac_3', 'cid_3','ss_3',
                        'mnc_4','lac_4', 'cid_4','ss_4',
                        'mnc_5','lac_5', 'cid_5','ss_5',
                        'mnc_6','lac_6', 'cid_6','ss_6',
                        'mnc_7','lac_7', 'cid_7','ss_7',
                        'mnc_8','lac_8', 'cid_8','ss_8',
                        'lon','lat','lon2','lat2',
                        'lon3','lat3','lon4','lat4',
                        'lon5','lat5','lon6','lat6',
                        'lon7','lat7','lon8','lat8',]
grid_size = 20

In [5]:
# 训练及测试
grider = RoadGrid(sf[['x', 'y']].values, grid_size)
sf['grid_id'] = grider.transform(sf[['x', 'y']].values, False)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X,Xe,Y,Ye = train_test_split(sf, sf[['x','y','grid_id']], test_size=0.2, random_state=200)
regr = RandomForestClassifier(n_estimators=10).fit(X[feature_cloumns_name], Y['grid_id'])
pred= regr.predict(Xe[feature_cloumns_name])
pred_loc = np.array([grider.grid_center[idx] for idx in pred])
err = [distance(p,t) for p, t in zip(pred_loc, Ye[['x','y']].values)]
print (np.median(err), np.mean(err), err[int(len(err)*0.9)])

185.15 575.699 1868.3


In [6]:
# 交叉验证
grider = RoadGrid(sf[['x', 'y']].values, grid_size)
sf['grid_id'] = grider.transform(sf[['x', 'y']].values, False)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
med, mea, nin = [], [], []
# 10-fold cross validation
for i in range(10):
    X,Xe,Y,Ye = train_test_split(sf, sf[['x','y','grid_id']], test_size=0.2, random_state=200)
    regr = RandomForestClassifier().fit(X[feature_cloumns_name], Y['grid_id'])
    pred= regr.predict(Xe[feature_cloumns_name])
    pred_loc = np.array([grider.grid_center[idx] for idx in pred])
    err = [distance(p,t) for p, t in zip(pred_loc, Ye[['x','y']].values)]
    err = sorted(err)
    med.append(np.median(err))
    mea.append(np.mean(err))
    nin.append(err[int(len(err)*0.9)])
    print (np.median(err), np.mean(err), err[int(len(err)*0.9)])
print (np.mean(med), np.mean(mea), np.mean(nin))

162.5 480.5299999999999 1319.8
164.5 493.689 1487.1
164.5 481.87700000000007 1359.9
148.5 582.075 1487.1
168.35 572.6340000000001 1524.9
163.6 524.8690000000001 1218.6
164.5 622.691 1863.8
135.35000000000002 479.18699999999995 1218.6
171.45 642.134 1711.6
164.5 564.322 1408.9
160.775 544.4008000000001 1460.03
